# Advent of Code 2022
## [Day 11: Monkey in the Middle](https://adventofcode.com/2022/day/11)

#### Load Data

In [1]:
import aocd
input_data = aocd.get_data(year=2022, day=11)
input_groups = [group.split("\n") for group in input_data.split("\n\n")]
input_groups[0]

['Monkey 0:',
 '  Starting items: 54, 98, 50, 94, 69, 62, 53, 85',
 '  Operation: new = old * 13',
 '  Test: divisible by 3',
 '    If true: throw to monkey 2',
 '    If false: throw to monkey 1']

In [2]:
import re

In [3]:
def parse_ints(line:str):
    return [int(i) for i in re.findall(r'(\d+)', line)]

parse_ints(input_groups[0][1])

[54, 98, 50, 94, 69, 62, 53, 85]

### Part 1

In [4]:
class Monkey:
    def __init__(self, lines: str):
        self.id, = parse_ints(lines[0])
        self.items = parse_ints(lines[1])
        self.operation = lines[2][19:]
        self.test_divisor, = parse_ints(lines[3])
        self.if_true, = parse_ints(lines[4])
        self.if_false, = parse_ints(lines[5])
        self.inspections = 0
        
    def __repr__(self):
        return f"Monkey {self.id} ({self.inspections}) {self.items}"

    def operate(self, old):
        return eval(self.operation)
    
    def throw_to(self, worry_level):
        if worry_level % self.test_divisor == 0:
            return self.if_true
        else:
            return self.if_false
        
    def take_turn(self, relief_factor=3, modulus=None):
        for worry_level in self.items:
            self.inspections += 1
            worry_level = self.operate(worry_level)
            worry_level //= relief_factor
            if modulus:
                worry_level = worry_level % modulus
            target = self.throw_to(worry_level)
            monkeys[target].items.append(worry_level)
        self.items = []
        
Monkey(input_groups[0]).__dict__

{'id': 0,
 'items': [54, 98, 50, 94, 69, 62, 53, 85],
 'operation': 'old * 13',
 'test_divisor': 3,
 'if_true': 2,
 'if_false': 1,
 'inspections': 0}

In [5]:
monkeys = [Monkey(g) for g in input_groups]
monkeys

[Monkey 0 (0) [54, 98, 50, 94, 69, 62, 53, 85],
 Monkey 1 (0) [71, 55, 82],
 Monkey 2 (0) [77, 73, 86, 72, 87],
 Monkey 3 (0) [97, 91],
 Monkey 4 (0) [78, 97, 51, 85, 66, 63, 62],
 Monkey 5 (0) [88],
 Monkey 6 (0) [87, 57, 63, 86, 87, 53],
 Monkey 7 (0) [73, 59, 82, 65]]

In [6]:
monkeys[5].take_turn()
monkeys

[Monkey 0 (0) [54, 98, 50, 94, 69, 62, 53, 85, 30],
 Monkey 1 (0) [71, 55, 82],
 Monkey 2 (0) [77, 73, 86, 72, 87],
 Monkey 3 (0) [97, 91],
 Monkey 4 (0) [78, 97, 51, 85, 66, 63, 62],
 Monkey 5 (1) [],
 Monkey 6 (0) [87, 57, 63, 86, 87, 53],
 Monkey 7 (0) [73, 59, 82, 65]]

In [7]:
def run_round(**kwargs):
    for monkey in monkeys:
        monkey.take_turn(**kwargs)

#### Part 1 Answer
Figure out which monkeys to chase by counting how many items they inspect over 20 rounds.  
**What is the level of monkey business after 20 rounds of stuff-slinging simian shenanigans?**

In [8]:
monkeys = [Monkey(g) for g in input_groups]
for _ in range(20):
    run_round()
monkeys

[Monkey 0 (323) [3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 675],
 Monkey 1 (302) [],
 Monkey 2 (333) [],
 Monkey 3 (318) [7378, 225, 225, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 Monkey 4 (55) [],
 Monkey 5 (314) [],
 Monkey 6 (25) [],
 Monkey 7 (337) []]

In [9]:
def monkey_business(monkeys):
    inspections = sorted([m.inspections for m in monkeys])
    return inspections[-1] * inspections[-2]
monkey_business(monkeys)

112221

---

### Part 2

In [10]:
monkeys = [Monkey(g) for g in input_groups]
for _ in range(20):
    run_round(relief_factor=1)
monkeys

[Monkey 0 (273) [53892346895282753309310478214642, 59712408315679146282377528668, 13149678514622588, 3472077778387243602206203369322899697198, 3472077778387243602206203369322899697198, 10303565507395642769907270316, 91769706111, 91769706111, 11417375036475, 70309713297, 12857646816140937, 44600529147, 17425633221495, 165223402063959, 157318971377469, 22943649707159, 12709492466457],
 Monkey 1 (155) [2833675514109104851624611488],
 Monkey 2 (309) [],
 Monkey 3 (280) [507955391644306930325229486, 1169823957396270422712689761068, 46247045998327466063114695213457965558480192910754308937438, 323241357713003570192386747424038482635425538582794, 3776260185870161992482131971479074409759484, 766635825704008610165336913014003455439, 1193006179457, 14153335874705, 17742026259542183, 17742026259542183, 226533231879449, 103300753871849],
 Monkey 4 (132) [11198678988554607191821461465314, 1489278800000820878784066661344308719304498134362, 20773888939950193955113079760, 110372212057522785177194185270

In [11]:
from functools import reduce

In [12]:
def get_modulus(monkeys):
    return reduce(lambda x,y:x*y, [m.test_divisor for m in monkeys])

get_modulus(monkeys)

9699690

In [13]:
monkeys = [Monkey(g) for g in input_groups]
for _ in range(20):
    run_round(relief_factor=1, modulus=get_modulus(monkeys))
monkeys

[Monkey 0 (273) [2327372, 2528788, 1311248, 3618488, 3618488, 2663326, 939021, 939021, 5733135, 6360177, 2659467, 1354527, 4340835, 7767999, 9657549, 2981159, 8058837],
 Monkey 1 (155) [4367678],
 Monkey 2 (309) [],
 Monkey 3 (280) [3651906, 4682208, 3367428, 2862664, 9333544, 8973329, 2507597, 4112135, 9273863, 9273863, 7932419, 5941469],
 Monkey 4 (132) [3096224, 7402242, 9693570, 668814, 8227014, 2583506],
 Monkey 5 (276) [],
 Monkey 6 (39) [],
 Monkey 7 (297) []]

#### Part 2 Answer
Worry levels are no longer divided by three after each item is inspected; you'll need to find another way to keep your worry levels manageable. Starting again from the initial state in your puzzle input, **what is the level of monkey business after 10000 rounds?**

In [14]:
monkeys = [Monkey(g) for g in input_groups]
for _ in range(10000):
    run_round(relief_factor=1, modulus=get_modulus(monkeys))
monkeys

[Monkey 0 (148544) [899746, 5018302, 1418297, 7669316, 7182232, 7182232, 5003991, 2881065, 59337, 7018185, 8429049, 3138309, 3101181, 3101181, 3296103, 8057769],
 Monkey 1 (59007) [],
 Monkey 2 (161943) [],
 Monkey 3 (156055) [8812698, 2369392, 3280354, 4762822, 7669312, 3670031, 5857931, 5857931, 6853757, 7194539, 1516607, 2045681, 2045681, 1356161, 4344965, 374921, 7150965],
 Monkey 4 (54734) [7789658, 6224094, 6071822],
 Monkey 5 (156056) [],
 Monkey 6 (5926) [],
 Monkey 7 (149165) []]

In [15]:
monkey_business(monkeys)

25272176808